In [80]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [81]:

cwd = os.getcwd()
args = {
    "dataset": "/data/training/simple",
    "plot": os.path.join(cwd, "plot"),
    "model": os.path.join("/data/training/hello-keras","dense.h5"),
    "label_bin": os.path.join("/data/training/hello-keras","labels.pkl")
}

In [90]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(args["dataset"])))
random.seed(42)
random.shuffle(imagePaths)


[INFO] loading images...


In [91]:
  
# loop over the input images
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    try:   
        # load the image, resize the image to be 32x32 pixels (ignoring
        # aspect ratio), flatten the image into 32x32x3=3072 pixel image
        # into a list, and store the image in the data list
        image = cv2.resize(image, (64, 64)).flatten()
        data.append(image)
    
        # extract the class label from the image path and update the
        # labels list
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)

    except Exception as e:
        print(imagePath)
        print(str(e))

In [93]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [97]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

In [97]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [105]:


# define the 3072-1024-512-3 architecture using Keras
model = Sequential()
model.add(Dropout(0.1, input_shape=(12288,)))
model.add(Dense(1024, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

In [106]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS = 150

# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

[INFO] training network...


In [110]:
# train the neural network
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=256)

Train on 3840 samples, validate on 960 samples
Epoch 1/150
 256/3840 [=>............................] - ETA: 0s - loss: 1.3538 - acc: 0.3203

 768/3840 [=====>........................] - ETA: 0s - loss: 1.3336 - acc: 0.3359

1536/3840 [===========>..................] - ETA: 0s - loss: 1.3324 - acc: 0.3262

2304/3840 [=================>............] - ETA: 0s - loss: 1.3353 - acc: 0.3242

3072/3840 [=======================>......] - ETA: 0s - loss: 1.3348 - acc: 0.3148

...

1024/3840 [=======>......................] - ETA: 0s - loss: 1.2942 - acc: 0.3643

1792/3840 [=============>................] - ETA: 0s - loss: 1.2929 - acc: 0.3705

2560/3840 [===================>..........] - ETA: 0s - loss: 1.2924 - acc: 0.3812

3328/3840 [=========================>....] - ETA: 0s - loss: 1.2930 - acc: 0.3810

3840/3840 [==============================] - 0s 87us/step - loss: 1.2941 - acc: 0.3812 - val_loss: 1.2938 - val_acc: 0.3750


In [123]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=256)
#report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_)
#print(report)

# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("plot") #args["plot"]

[INFO] evaluating network...


In [124]:
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save(args["model"])
f = open(args["label_bin"], "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network and label binarizer...
